## Imports

In [28]:
import pandas as pd
import numpy as np
import folium
import googlemaps
import os

## Data read in

In [4]:
df = pd.read_csv('waterfront_reduction_miami_beach_data.csv')

## Geocode

In [9]:
df['geo_address'] = df['Address'] + ' ' + df['City Name']

In [7]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [10]:
def geocode(add):
    g = gmaps_key.geocode(add)
    lat = g[0]["geometry"]["location"]["lat"]
    lng = g[0]["geometry"]["location"]["lng"]
    return (lat, lng)

df['geocoded'] = df['geo_address'].apply(geocode)

In [11]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].str.strip('()').str.split(', ', expand=True)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

## Color code active vs. closed listings

In [14]:
# use numpy to assign values to the 'COLOR' column
df['COLOR'] = np.where(df['Status'] == 'Active', 'orange', 'blue')

In [21]:
df[['Status','COLOR']].head(5)

,Status,COLOR
0,Active,orange
1,Closed,blue
2,Closed,blue
3,Closed,blue
4,Active,orange


## Remove nans

In [37]:
df = df.fillna('Not provided')

## Set up popups

In [38]:
def popup_html(row):
    status = row['Status']
    address = row['Address']
    list_price_change_date = row['List Price Chg Date']
    orig_list_price = row['Original List Price']
    cdom = row['CDOM (Days on Market)']
    year_built = row['Year Built']
    days_to_closing_from_price_change = row['Days to Closing from Price Change']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Status: </strong>{}'''.format(status) + '''<br>
    <strong>Address: </strong>{}'''.format(address) + '''<br>
    <strong>Listing Price Change Date: </strong>{}'''.format(list_price_change_date) + '''<br>
    <strong>Original Listing Price: </strong>{}'''.format(orig_list_price) + '''<br>
    <strong>Days on Market: </strong>{}'''.format(cdom) + '''<br>
    <strong>Year Built: </strong>{}'''.format(year_built) + '''<br>
    <strong>Days to Closing from Price Change: </strong>{}'''.format(days_to_closing_from_price_change) + '''<br>
    </html>
    '''
    return html

In [39]:
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=10)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Price Reductions ')


### Create map container ###
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=11.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='Closed Listings')
fg_orange = folium.FeatureGroup(name='Active Listings')

for index, row in df.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['lat'], row['lon']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['lat'], row['lon']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
            
# Display map
m

In [40]:
m.save('index.html')

## Map URL Snagger

In [29]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/miami_beach_price_reductions
